# Install Libraries

In [1]:
library(tidyverse)
library(tidycensus)
library(readr)
library(dplyr)
library(ggplot2)
library(tidyr)
library(corrplot)
library(RCurl)
library(maps)
library(usmap)
library(lubridate)
library("httr")
library("RCurl")
library(choroplethrMaps)
library(tidycensus)
library(knitr)  # for kable
library(maps)
library(readr)
library(dplyr)
library(tidyverse)
library(choroplethr)
library("usmap")
library("reshape2")
library('openintro')
library(zoo)
census_api_key("5dc3854fc03ae75783dcbdbe2a9db1509f250cbf")

Warning message:
"package 'tidyverse' was built under R version 3.6.3"-- Attaching packages --------------------------------------- tidyverse 1.3.0 --
v ggplot2 3.3.0     v purrr   0.3.3
v tibble  3.0.0     v dplyr   0.8.5
v tidyr   1.0.2     v stringr 1.4.0
v readr   1.3.1     v forcats 0.5.0
Warning message:
"package 'ggplot2' was built under R version 3.6.3"Warning message:
"package 'tibble' was built under R version 3.6.3"Warning message:
"package 'tidyr' was built under R version 3.6.3"Warning message:
"package 'readr' was built under R version 3.6.3"Warning message:
"package 'purrr' was built under R version 3.6.3"Warning message:
"package 'dplyr' was built under R version 3.6.3"Warning message:
"package 'stringr' was built under R version 3.6.3"Warning message:
"package 'forcats' was built under R version 3.6.3"-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
Warning mess

# Download Census Data

In [2]:
Manufacturing <- get_acs(geography = "county", variables = "DP03_0035PE" , year = 2017)
Manufacturing <- data.frame(Manufacturing$GEOID,Manufacturing$NAME,Manufacturing$estimate)
Manufacturing<- rename(Manufacturing, c("countyfips"= "Manufacturing.GEOID","NAME" = "Manufacturing.NAME","Manufacturing"="Manufacturing.estimate"))

Getting data from the 2013-2017 5-year ACS
Using the ACS Data Profile


In [3]:
pop <- get_acs(geography = "county", variables = "DP05_0001E" , year = 2017)
pop <- data.frame(pop$GEOID,pop$NAME,pop$estimate)
pop<- rename(pop, c("countyfips"= "pop.GEOID","NAME" = "pop.NAME","pop"="pop.estimate"))

Getting data from the 2013-2017 5-year ACS
Using the ACS Data Profile


In [4]:
Manufacturing <- full_join(Manufacturing, pop, by.x = "countyfips",all.x= TRUE)

Joining, by = c("countyfips", "NAME")


In [5]:
#turn fips into a number
Manufacturing[,c(1)] <- as.numeric(as.character(Manufacturing[,c(1)]))

In [6]:
Manufacturing$decile <- ntile(Manufacturing$Manufacturing, 5)

# Download Covid Economic Data

In [7]:
#Credit card data
credit <- read.csv("https://raw.githubusercontent.com/OpportunityInsights/EconomicTracker/main/data/Affinity%20-%20County%20-%20Weekly.csv")
credit<- rename(credit, c("day"= "day_endofweek"))

In [8]:
charlotte<-rbind(unique(credit$countyfips))
ncol(charlotte)

[1] 1774

In [9]:
#Cases per 100k
cases <- read.csv("https://raw.githubusercontent.com/OpportunityInsights/EconomicTracker/main/data/COVID%20Cases%20-%20County%20-%20Daily.csv")

In [11]:
ben<-rbind(unique(cases$countyfips))
ncol(ben)

[1] 3142

In [12]:
#deaths per 100k
deaths <- read.csv("https://raw.githubusercontent.com/OpportunityInsights/EconomicTracker/main/data/COVID%20Deaths%20-%20County%20-%20Daily.csv")

In [13]:
#Google Mobility
mobility <- read.csv("https://raw.githubusercontent.com/OpportunityInsights/EconomicTracker/main/data/Google%20Mobility%20-%20County%20-%20Daily.csv")

In [14]:
mobility[mobility == "."] <- NA
mobility= select(mobility, year, month, day,countyfips,gps_workplaces)

In [15]:
#Business Revenue
revenue <- read.csv("https://raw.githubusercontent.com/OpportunityInsights/EconomicTracker/main/data/Womply%20Revenue%20-%20County%20-%20Daily.csv")

In [16]:
marmite<-rbind(unique(revenue$countyfips))
ncol(marmite)

[1] 2118

In [17]:
#geoid
geoid <- read.csv("https://raw.githubusercontent.com/OpportunityInsights/EconomicTracker/main/data/GeoIDs%20-%20County.csv")

In [18]:
#geoid
gdp <- read.csv("gdp_weights.csv")
gdp<- rename(gdp, c("statename"= "ï..State"))
geoid <- geoid %>% left_join(gdp, by=c("statename"))

Warning message:
"Column `statename` joining factors with different levels, coercing to character vector"

# Merge Data

In [19]:
Manufacturing <- Manufacturing %>% left_join(geoid, by=c("countyfips"))

In [20]:
credit_1 <- credit %>% left_join(Manufacturing, by=c("countyfips"))
credit_1$Date <- as.Date(with(credit_1, paste(year, month, day,sep="-")), "%Y-%m-%d")
credit_1<-na.omit(credit_1)

#sum the total weight of deciles
credit_1weights <- aggregate(credit_1$pop, by=list(credit_1$decile,credit_1$Date), FUN=sum)
#rename the weight
credit_1weights<- rename(credit_1weights, c("decile"= "Group.1", "Date" = "Group.2"))
#put into the database
credit_1 <- credit_1 %>% left_join(credit_1weights, by=c("Date","decile"))
#create the weight
credit_1$weight <- credit_1[, "pop"] / credit_1[, "x"]

In [21]:
#credit_1 <- aggregate(credit_1$weight, by=list(credit_1$decile,credit_1$Date), FUN=sum)

In [22]:
cases_1 <- cases %>% left_join(Manufacturing, by=c("countyfips"))
cases_1$Date <- as.Date(with(cases_1, paste(year, month, day,sep="-")), "%Y-%m-%d")
cases_1<-na.omit(cases_1)

#sum the total weight of deciles
cases_1weights <- aggregate(cases_1$pop, by=list(cases_1$decile,cases_1$Date), FUN=sum)
#rename the weight
cases_1weights<- rename(cases_1weights, c("decile"= "Group.1", "Date" = "Group.2"))
#put into the database
cases_1 <- cases_1 %>% left_join(cases_1weights, by=c("Date","decile"))
#create the weight
cases_1$weight <- cases_1[, "pop"] / cases_1[, "x"]

In [23]:
deaths_1 <- deaths %>% left_join(Manufacturing, by=c("countyfips"))
deaths_1$Date <- as.Date(with(deaths_1, paste(year, month, day,sep="-")), "%Y-%m-%d")
deaths_1<-na.omit(deaths_1)

#sum the total weight of deciles
deaths_1weights <- aggregate(deaths_1$pop, by=list(deaths_1$decile,deaths_1$Date), FUN=sum)
#rename the weight
deaths_1weights<- rename(deaths_1weights, c("decile"= "Group.1", "Date" = "Group.2"))
#put into the database
deaths_1 <- deaths_1 %>% left_join(deaths_1weights, by=c("Date","decile"))
#create the weight
deaths_1$weight <- deaths_1[, "pop"] / deaths_1[, "x"]

In [24]:
#deaths_1 <- aggregate(deaths_1$weight, by=list(deaths_1$decile,deaths_1$Date), FUN=sum)

In [25]:
mobility_1 <- mobility %>% left_join(Manufacturing, by=c("countyfips"))
mobility_1$Date <- as.Date(with(mobility_1, paste(year, month, day,sep="-")), "%Y-%m-%d")
mobility_1<-na.omit(mobility_1)

#sum the total weight of deciles
mobility_1weights <- aggregate(mobility_1$pop, by=list(mobility_1$decile,mobility_1$Date), FUN=sum)
#rename the weight
mobility_1weights<- rename(mobility_1weights, c("decile"= "Group.1", "Date" = "Group.2"))
#put into the database
mobility_1 <- mobility_1 %>% left_join(mobility_1weights, by=c("Date","decile"))
#create the weight
mobility_1$weight <- mobility_1[, "pop"] / mobility_1[, "x"]

In [26]:
revenue_1 <- revenue %>% left_join(Manufacturing, by=c("countyfips"))
revenue_1$Date <- as.Date(with(revenue_1, paste(year, month, day,sep="-")), "%Y-%m-%d")
revenue_1<-na.omit(revenue_1)

#sum the total weight of deciles
revenue_1weights <- aggregate(revenue_1$pop, by=list(revenue_1$decile,revenue_1$Date), FUN=sum)
#rename the weight
revenue_1weights<- rename(revenue_1weights, c("decile"= "Group.1", "Date" = "Group.2"))
#put into the database
revenue_1 <- revenue_1 %>% left_join(revenue_1weights, by=c("Date","decile"))
#create the weight
revenue_1$weight <- revenue_1[, "pop"] / revenue_1[, "x"]

# Apply weights

In [27]:
credit_1$spend_all <- as.numeric(as.character(credit_1$spend_all))
credit_1$spend_all_weight <- credit_1$spend_all* credit_1$weight

cases_1$case_rate <- as.numeric(as.character(cases_1$case_rate))
cases_1$spend_all_weight <- cases_1$case_rate* cases_1$weight

deaths_1$death_rate <- as.numeric(as.character(deaths_1$death_rate))
deaths_1$spend_all_weight <- deaths_1$death_rate* deaths_1$weight

mobility_1$gps_workplaces <- as.numeric(as.character(mobility_1$gps_workplaces))
mobility_1$spend_all_weight <- mobility_1$gps_workplaces* mobility_1$weight

revenue_1$revenue_all <- as.numeric(as.character(revenue_1$revenue_all))
revenue_1$spend_all_weight <- revenue_1$revenue_all* credit_1$weight

Warning message in eval(expr, envir, enclos):
"NAs introduced by coercion"Warning message in eval(expr, envir, enclos):
"NAs introduced by coercion"Warning message in revenue_1$revenue_all * credit_1$weight:
"longer object length is not a multiple of shorter object length"

In [28]:
tail(cases_1)

,year,month,day,countyfips,case_rate,new_case_rate,NAME,Manufacturing,pop,decile,...,statefips,stateabbrev,county_pop2019,GDP_Contribution,Region,GDP_region,Date,x,weight,spend_all_weight
576283,2020,7,22,56035,183.0,11.6,"Sublette County, Wyoming",2.0,10037,1,...,56,WY,9831,0.002,Rocky Mountain,0.006452,2020-07-22,64439536,1.557584e-04,0.028503790
576284,2020,7,22,56037,475.0,14.8,"Sweetwater County, Wyoming",6.8,44527,2,...,56,WY,42343,0.002,Rocky Mountain,0.006452,2020-07-22,92548011,4.811233e-04,0.228533545
576285,2020,7,22,56039,997.0,39,"Teton County, Wyoming",1.4,22923,1,...,56,WY,23464,0.002,Rocky Mountain,0.006452,2020-07-22,64439536,3.557288e-04,0.354661632
576286,2020,7,22,56041,1107.0,11.3,"Uinta County, Wyoming",5.2,20758,1,...,56,WY,20226,0.002,Rocky Mountain,0.006452,2020-07-22,64439536,3.221314e-04,0.356599495
576287,2020,7,22,56043,564.0,3.66,"Washakie County, Wyoming",9.9,8253,3,...,56,WY,7805,0.002,Rocky Mountain,0.006452,2020-07-22,89325098,9.239285e-05,0.052109565
576288,2020,7,22,56045,57.7,2.06,"Weston County, Wyoming",6.2,7117,2,...,56,WY,6927,0.002,Rocky Mountain,0.006452,2020-07-22,92548011,7.690063e-05,0.004437166


In [29]:
#sum up each count by date
credit_weight <- aggregate(credit_1$spend_all_weight, by=list(credit_1$Date,credit_1$decile), sum)
credit_weight<- rename(credit_weight, c("Date"= "Group.1","decile"="Group.2","credit"="x"))


cases_weight <- aggregate(cases_1$spend_all_weight, by=list(cases_1$Date,cases_1$decile), sum)
cases_weight<- rename(cases_weight, c("Date"= "Group.1","decile"="Group.2","cases"="x"))


deaths_weight <- aggregate(deaths_1$spend_all_weight, by=list(deaths_1$Date,deaths_1$decile), sum)
deaths_weight<- rename(deaths_weight, c("Date"= "Group.1","decile"="Group.2","deaths"="x"))


mobility_weight <- aggregate(mobility_1$spend_all_weight, by=list(mobility_1$Date,mobility_1$decile), sum)
mobility_weight<- rename(mobility_weight, c("Date"= "Group.1","decile"="Group.2","mobility"="x"))


revenue_weight <- aggregate(revenue_1$spend_all_weight, by=list(revenue_1$Date,revenue_1$decile), sum)
revenue_weight<- rename(revenue_weight, c("Date"= "Group.1","decile"="Group.2","revenue"="x"))

In [30]:
tail(cases_weight,250)

,Date,decile,cases
671,2020-05-18,4,315.8052
672,2020-05-19,4,321.1032
673,2020-05-20,4,327.5370
674,2020-05-21,4,334.5552
675,2020-05-22,4,341.6151
676,2020-05-23,4,348.0734
677,2020-05-24,4,353.8415
678,2020-05-25,4,358.6165
679,2020-05-26,4,363.8075
680,2020-05-27,4,369.1168


# Merge into one and put in CSV

In [31]:
final <- cases_weight %>% left_join(credit_weight, by=c("Date","decile"))
final <- final %>% left_join(deaths_weight, by=c("Date","decile"))
final <- final %>% left_join(mobility_weight, by=c("Date","decile"))
final <- final %>% left_join(revenue_weight, by=c("Date","decile"))

In [33]:
write.csv(final,"weighted_econ_data.csv")

In [ ]:
tail(final,50)

In [35]:
deaths_check <- read.csv("https://raw.githubusercontent.com/OpportunityInsights/EconomicTracker/main/data/COVID%20Deaths%20-%20State%20-%20Daily.csv")
state_check <- read.csv("https://raw.githubusercontent.com/OpportunityInsights/EconomicTracker/main/data/GeoIDs%20-%20State.csv")

In [36]:
deaths_check$Date <- as.Date(with(deaths_check, paste(year, month, day,sep="-")), "%Y-%m-%d")

In [39]:
deaths_check <- deaths_check %>% left_join(state_check, by=c("statefips"))

In [40]:
head(deaths_check)

year,month,day,statefips,death_rate,new_death_rate,Date,statename,stateabbrev,state_pop2019
2020,1,21,1,0,.,2020-01-21,Alabama,AL,4903185
2020,1,21,2,0,.,2020-01-21,Alaska,AK,731545
2020,1,21,4,0,.,2020-01-21,Arizona,AZ,7278717
2020,1,21,5,0,.,2020-01-21,Arkansas,AR,3017804
2020,1,21,6,0,.,2020-01-21,California,CA,39512223
2020,1,21,8,0,.,2020-01-21,Colorado,CO,5758736


In [42]:
write.csv(deaths_check,"deaths_check.csv")